In [178]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import unit_norm
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
from tensorflow.keras.models import save_model

In [2]:
df = pd.read_csv('../speech_emotion_reco/data/merged_dataset.csv')
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

In [100]:
df_train.head()

,observation,path,emotion
0,8105_stretch,../speech_emotion_reco/data/images_augmented/s...,2
1,11516_shift,../speech_emotion_reco/data/images_augmented/s...,3
2,8889_pitch,../speech_emotion_reco/data/images_augmented/p...,4
3,1741_shift,../speech_emotion_reco/data/images_augmented/s...,2
4,6984_pitch,../speech_emotion_reco/data/images_augmented/p...,4


In [102]:
df_train.emotion.value_counts()

3    6735
4    6729
0    6713
1    6654
2    6644
5    6570
Name: emotion, dtype: int64

In [101]:
df.head()

,Unnamed: 0,gender,emotion,path,duration
0,0,male,sad,../speech_emotion_reco/data/savee/JK_sa01.wav,4.511837
1,1,male,sad,../speech_emotion_reco/data/savee/JK_sa15.wav,6.058730
2,2,male,neutral,../speech_emotion_reco/data/savee/DC_n13.wav,2.788889
3,3,male,surprise,../speech_emotion_reco/data/savee/DC_su09.wav,3.433968
4,4,male,neutral,../speech_emotion_reco/data/savee/DC_n07.wav,4.051791


In [107]:
def load_data(images_path):
    classes = {'happy': 0, 
          'sad': 1,
          'fear': 2,
          'disgust': 3,
          'angry': 4,
          'neutral': 5,
          'surprise': 6,
          'unknown': 7}
    
    imgs = []
    labels = []
    
    for path in images_path:
        index = int(path.replace('.png','').replace(data_path,'').split('_')[1])
        image = Image.open(path).convert('RGB')
        resized_image = image.resize((256,256))
        imgs.append(np.array(resized_image))
        labels.append(classes[df.loc[index, 'emotion']])

    X = np.array(imgs)
    y = to_categorical(np.array(labels), num_classes=6)
    
    return X,y

In [108]:
X_test, y_test = load_data(df_test.path.values)

In [116]:
X_train, y_train = load_data(df_train.path.values)

In [155]:
y_train[:10]

array([[0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [156]:
def load_model():
    model = Sequential()
    model.add(layers.Conv2D(120, kernel_size=11, activation="relu", strides=4, input_shape=(256,256,3), kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(256, kernel_size=5, activation="relu", strides=1,kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Conv2D(384, kernel_size=3, activation="relu", kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.MaxPooling2D(3, strides=2))
    model.add(layers.Flatten())
    model.add(layers.Dense(2048, activation='relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2048, activation='relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

In [164]:
def build_model():  
    model = load_model()
    model.compile(loss='categorical_crossentropy',
                  optimizer = 'adam',
                  metrics=['accuracy'])
    return model

In [165]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    validation_split=0.2,
                    epochs=100, 
                    batch_size=100,
                    callbacks=[es],
                    verbose=1)

Epoch 1/100


2021-09-01 17:48:36.223766: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


321/321 [==============================] - ETA: 0s - loss: 106.4909 - accuracy: 0.2533

2021-09-01 17:53:01.223815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


321/321 [==============================] - 287s 883ms/step - loss: 106.4909 - accuracy: 0.2533 - val_loss: 60.4799 - val_accuracy: 0.3422
Epoch 2/100
321/321 [==============================] - 296s 923ms/step - loss: 53.6832 - accuracy: 0.3494 - val_loss: 47.6900 - val_accuracy: 0.4139
Epoch 3/100
321/321 [==============================] - 308s 958ms/step - loss: 43.2046 - accuracy: 0.4126 - val_loss: 39.0452 - val_accuracy: 0.4485
Epoch 4/100
321/321 [==============================] - 343s 1s/step - loss: 35.5656 - accuracy: 0.4661 - val_loss: 32.3096 - val_accuracy: 0.5049
Epoch 5/100
321/321 [==============================] - 384s 1s/step - loss: 29.6924 - accuracy: 0.4964 - val_loss: 27.1156 - val_accuracy: 0.4981
Epoch 6/100
321/321 [==============================] - 362s 1s/step - loss: 24.9706 - accuracy: 0.5150 - val_loss: 22.8761 - val_accuracy: 0.5197
Epoch 7/100
321/321 [==============================] - 350s 1s/step - loss: 21.0813 - accuracy: 0.5312 - val_loss: 19.3427 - v

In [169]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)

537/537 - 50s - loss: 1.2834 - accuracy: 0.6495


In [179]:
save_model(
    model, '../speech_emotion_reco/CNN_model', overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
)

INFO:tensorflow:Assets written to: ../speech_emotion_reco/CNN_model/assets


In [180]:
loaded_model = tf.keras.models.load_model('../speech_emotion_reco/CNN_model')

In [181]:
assert np.allclose(model.predict(X_test), loaded_model.predict(X_test))

2021-09-02 19:31:46.572153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-09-02 19:32:24.194281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [162]:
loss, acc = model_test.evaluate(X_test, y_test, verbose=2)

2021-09-01 17:46:15.118404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


537/537 - 41s - loss: 49.0360 - accuracy: 0.1657


In [121]:
y_pred = model.predict(X_test)

In [138]:
pred_mlp = pd.read_csv('../speech_emotion_reco/data/test_proba_raph.csv')

In [140]:
pred_mlp = pred_mlp.set_index('observation')

In [141]:
classes = {'happy': 0, 
          'sad': 1,
          'fear': 2,
          'disgust': 3,
          'angry': 4,
          'neutral': 5,
          'surprise': 6,
          'unknown': 7}
index = 0 
for column in y_pred.T:
    pred_mlp[list(classes.keys())[index] + "_raph"] = column
    index += 1

In [147]:
df_test

,observation,path,emotion
0,0_original,../speech_emotion_reco/data/images_augmented/o...,1
1,0_stretch,../speech_emotion_reco/data/images_augmented/s...,1
2,10000_original,../speech_emotion_reco/data/images_augmented/o...,2
3,10000_pitch,../speech_emotion_reco/data/images_augmented/p...,2
4,10001_noise,../speech_emotion_reco/data/images_augmented/n...,2
...,...,...,...
17157,9999_shift,../speech_emotion_reco/data/images_augmented/s...,0
17158,9999_stretch,../speech_emotion_reco/data/images_augmented/s...,0
17159,999_pitch,../speech_emotion_reco/data/images_augmented/p...,4
17160,99_original,../speech_emotion_reco/data/images_augmented/o...,2


In [142]:
pred_mlp

,angry,disgust,fear,happy,neutral,sad,happy_raph,sad_raph,fear_raph,disgust_raph,angry_raph,neutral_raph
observation,,,,,,,,,,,,
0_original,2.291376e-07,6.642547e-08,7.583450e-14,5.367126e-15,3.795053e-10,9.999998e-01,0.019691,0.232391,0.085794,0.227592,0.013517,0.421015
0_stretch,4.400612e-09,4.845263e-05,9.796351e-10,3.116841e-12,3.912250e-04,9.985856e-01,0.021747,0.272781,0.118796,0.243764,0.013249,0.329663
10000_original,3.302859e-02,1.601639e-02,9.463198e-01,2.838790e-03,1.154285e-02,5.747730e-03,0.052651,0.381382,0.312122,0.157792,0.007922,0.088132
10000_pitch,2.075523e-02,5.564900e-01,3.435738e-01,8.536743e-03,2.496596e-01,1.189899e-01,0.011009,0.635718,0.209856,0.098195,0.002626,0.042596
10001_noise,9.921627e-01,4.149681e-05,1.331815e-03,6.289357e-03,7.248833e-10,1.193373e-07,0.169900,0.032887,0.108354,0.237867,0.421930,0.029062
...,...,...,...,...,...,...,...,...,...,...,...,...
9999_shift,7.964782e-06,1.432951e-08,4.564824e-06,9.999981e-01,1.777097e-09,8.359679e-09,0.492319,0.018659,0.053059,0.094926,0.229721,0.111317
9999_stretch,1.456020e-04,1.721675e-08,4.429794e-06,9.999564e-01,1.235610e-07,2.388654e-09,0.656049,0.006694,0.048226,0.049586,0.223210,0.016233
999_pitch,1.319471e-06,2.036677e-10,7.433832e-04,9.999913e-01,7.801448e-15,2.210580e-13,0.260744,0.026404,0.087453,0.159874,0.459900,0.005625


In [143]:
pred_mlp.to_csv('test_combined_proba.csv')

In [144]:
!pwd

/Users/raphaelvoortman/code/caronarthur/speech_emotion_reco/notebooks


In [145]:
!ls

Arthur-model.ipynb                      mateo_first_neural_network.ipynb
Arthur_data_exploration_featuring.ipynb mateo_neural_network.ipynb
Arthur_searching_model.ipynb            mateo_saving_spect_as_image.ipynb
EDA.ipynb                               merged_dataset.csv
Mateo_data_augmentation.ipynb           model.joblib
Untitled.ipynb                          raph_NN_model.ipynb
df_test                                 raph_data_merge.ipynb
df_test.csv                             raph_first_DL_model_VGG16.ipynb
df_train                                raph_transform_images_to_array.ipynb
df_train.csv                            spec.png
mateo_data_exploration.ipynb            test_combined_proba.csv


In [146]:
!open .

In [182]:
import joblib
model_mateo = joblib.load('../speech_emotion_reco/data/model_mateo.joblib')

/Users/raphaelvoortman/.pyenv/versions/3.8.6/envs/lewagon_tf/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/raphaelvoortman/.pyenv/versions/3.8.6/envs/lewagon_tf/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [188]:
example = pd.read_csv('../speech_emotion_reco/data/exemple.csv')

In [190]:
example.drop(columns='labels', inplace=True)

In [191]:
model_mateo.predict(example)

array([[0, 0, 0, 0, 0, 1]])